# Structured Bayesian Optimisation with Pyro (2D Eggholder function)

In [ ]:
import numpy as np
import torch
import pyro

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

In [ ]:
print(torch.__version__)
print(pyro.__version__)

In [ ]:
torch.random.manual_seed(555)

# Objective function 

In [ ]:
def eggholder(x):
    
    x1 = x[...,0]
    x2 = x[...,1]
    
    return -((x2 + 47) * torch.sin(torch.sqrt(torch.abs(x2 + x1/2 + 47))) - x1 * torch.sin(torch.sqrt(torch.abs(x1 - x2 + 47))))
    

# Visualisation

In [ ]:
const_x1_min = -512
const_x2_min = -512
const_x1_max = 512
const_x2_max = 512

steps = 1000
strides = 100

X1 = torch.linspace(const_x1_min, const_x1_max, steps)
X2 = torch.linspace(const_x2_min, const_x2_max, steps)

X1_mesh, X2_mesh = torch.meshgrid(X1, X2)

Z_mesh = eggholder(torch.stack((X1_mesh, X2_mesh), dim=2))


fig = plt.figure()
ax = plt.axes(projection='3d')
ax.contour3D(X1_mesh, X2_mesh, Z_mesh, 50, cmap='binary')
ax.set_xlabel('x1')
ax.set_ylabel('x2')
ax.set_zlabel('y');

In [ ]:
plt.contour(X1_mesh, X2_mesh, Z_mesh, strides)

# Initial (training) data

In [ ]:
N_points = 10

X = torch.rand(N_points, 2)*15 + torch.FloatTensor([-5, 0])

y = eggholder(X)